In [24]:
import numpy as np

sups = [7, 23, 57, 83, 91, 110, 9, 20, 89, 5, 21, 25, 27, 58, 64, 66, 75, 84, 86, 87, 90, 101, 111, 119, 16, 42, 
         85, 102, 108, 40, 3, 31, 26, 30, 37, 50, 52, 68, 79, 92, 112]

cores = [18, 19, 23, 49, 59, 73, 78, 99, 104, 1, 6, 8, 9, 10, 12, 32, 35, 46, 48, 70, 72, 80, 89, 95, 106, 114, 
         120, 17, 21, 25, 34, 53, 75, 28, 42, 54, 60, 69, 71, 77, 81, 102, 4, 11, 15, 41, 44, 47, 56, 61, 63, 67, 
         82, 93, 94, 109, 113, 36, 39, 43, 52, 74, 76]

idToHero = {1: 'antimage', 2: 'axe', 3: 'bane', 4: 'bloodseeker', 5: 'crystal_maiden', 6: 'drow_ranger', 
            7: 'earthshaker', 8: 'juggernaut', 9: 'mirana', 11: 'nevermore', 10: 'morphling', 12: 'phantom_lancer', 
            13: 'puck', 14: 'pudge', 15: 'razor', 16: 'sand_king', 17: 'storm_spirit', 18: 'sven', 19: 'tiny', 
            20: 'vengefulspirit', 21: 'windrunner', 22: 'zuus', 23: 'kunkka', 25: 'lina', 31: 'lich', 26: 'lion', 
            27: 'shadow_shaman', 28: 'slardar', 29: 'tidehunter', 30: 'witch_doctor', 32: 'riki', 33: 'enigma', 
            34: 'tinker', 35: 'sniper', 36: 'necrolyte', 37: 'warlock', 38: 'beastmaster', 39: 'queenofpain', 
            40: 'venomancer', 41: 'faceless_void', 42: 'skeleton_king', 43: 'death_prophet', 44: 'phantom_assassin',
            45: 'pugna', 46: 'templar_assassin', 47: 'viper', 48: 'luna', 49: 'dragon_knight', 50: 'dazzle', 51: 
            'rattletrap', 52: 'leshrac', 53: 'furion', 54: 'life_stealer', 55: 'dark_seer', 56: 'clinkz', 
            57: 'omniknight', 58: 'enchantress', 59: 'huskar', 60: 'night_stalker', 61: 'broodmother', 
            62: 'bounty_hunter', 63: 'weaver', 64: 'jakiro', 65: 'batrider', 66: 'chen', 67: 'spectre', 
            69: 'doom_bringer', 68: 'ancient_apparition', 70: 'ursa', 71: 'spirit_breaker', 72: 'gyrocopter', 
            73: 'alchemist', 74: 'invoker', 75: 'silencer', 76: 'obsidian_destroyer', 77: 'lycan', 
            78: 'brewmaster', 79: 'shadow_demon', 80: 'lone_druid', 81: 'chaos_knight', 82: 'meepo', 83: 'treant', 
            84: 'ogre_magi', 85: 'undying', 86: 'rubick', 87: 'disruptor', 88: 'nyx_assassin', 89: 'naga_siren', 
            90: 'keeper_of_the_light', 91: 'wisp', 92: 'visage', 93: 'slark', 94: 'medusa', 95: 'troll_warlord', 
            96: 'centaur', 97: 'magnataur', 98: 'shredder', 99: 'bristleback', 100: 'tusk', 101: 'skywrath_mage', 
            102: 'abaddon', 103: 'elder_titan', 104: 'legion_commander', 106: 'ember_spirit', 107: 'earth_spirit', 
            109: 'terrorblade', 110: 'phoenix', 111: 'oracle', 105: 'techies', 112: 'winter_wyvern', 
            113: 'arc_warden', 108: 'abyssal_underlord', 114: 'monkey_king', 120: 'pangolier', 119: 'dark_willow'}

import time

#mirana=9, bane=4 sinergia=0.0315556044752
#faceless void=41, phantom assassin=44 sinergia=-0.0205830863765
# earthshaker=7, phantom lancer=12



print(len(supps))

41


In [25]:
print(wr_counters[7,12] - wr_counters[12,7] - wr_pairs[7,7] + wr_pairs[12,12])

# print(syn_table
#       [41,44])

winrates = [(wr_pairs[hero, hero], hero) for hero in sups]
print(winrates)

0.0545901751471
[(0.48900262245156922, 7), (0.46672439017108053, 23), (0.51391769457482805, 57), (0.53612167300380231, 83), (0.4004744958481613, 91), (0.54376556215284433, 110), (0.51600393125614263, 9), (0.54798683210939481, 20), (0.45394363533194626, 89), (0.53160188101298589, 5), (0.48386816999132698, 21), (0.43096253926578837, 25), (0.51713284212032329, 27), (0.45039018952062432, 58), (0.49843937575030012, 64), (0.48002049180327871, 66), (0.52119328032522272, 75), (0.52144590004852009, 84), (0.45219983552631576, 86), (0.46994274809160308, 87), (0.47601615753597576, 90), (0.49938583321959873, 101), (0.46433483985065827, 111), (0.48744419642857145, 119), (0.45695005121407484, 16), (0.5539981044545319, 42), (0.54382451120648545, 85), (0.52095808383233533, 102), (0.55081085486105452, 108), (0.47657605552342397, 40), (0.46423864861626934, 3), (0.52754982415005858, 31), (0.4826878377154587, 26), (0.49189054726368159, 30), (0.55118408588569623, 37), (0.49245999769770921, 50), (0.448702830

In [71]:
bom1 = [10, 77, 96, 7, 101]
bom2 = [93, 27, 14, 49, 108]


# winrates.sort()
# print(winrates)

# print(idToHero[winrates[-6][1]], winrates[-6][1])
# print(idToHero[winrates[-7][1]], winrates[-7][1])
# print(idToHero[winrates[-8][1]], winrates[-8][1])
# print(idToHero[winrates[-4][1]], winrates[-4][1])
# print(idToHero[winrates[-9][1]], winrates[-9][1])


# sups = [110,83,5,20,31]


# time1 = [1, 60, 83, 85, 43]
# time2 = [17, 42, 108, 68, 31]
# # heroes = [71,42,6,81,4]
winrate = [wr_pairs[hero, hero] for hero in bom2]
avg = np.mean(winrate)
print(avg)

# sups = [110, 37, 83, 20, 5]
# cores = [71,42,6,81,4]

# time bom: Morphling, Lycanthrope, Centaur, Elder Titan e Skywrath Mage

# time pain: Morphling, Ember Spirit, Warlock, Omniknight e Tusk
# pain = [10, 106, 37, 57, 100]

print([idToHero[id] for id in bom1])
print([idToHero[id] for id in bom2])
# print([idToHero[id] for id in time2])

#time 1: Anti-Mage, Night Stalker, Treant, Undying e Death Prophet
#time 2: Storm Spirit, Wraith King, Underlord, Ancient Apparation e Lich

0.524457549319
['morphling', 'lycan', 'centaur', 'earthshaker', 'skywrath_mage']
['slark', 'shadow_shaman', 'pudge', 'dragon_knight', 'abyssal_underlord']


In [154]:
def gen_input(rad, dire):
    input = np.zeros((121 + 75))
    for hero in  rad:
        input[hero] = 1
    for hero in dire:
        input[hero] = -1
    
    counter = 121
    
    for hero1 in rad:
            for hero2 in rad:
                input[counter] = wr_pairs[hero1, hero2]
                counter += 1
              
  
    for hero1 in dire:
            for hero2 in dire:
                input[counter] = -wr_pairs[hero1, hero2]
                counter += 1
   
    for hero1 in rad:
            for hero2 in dire:
                input[counter] = wr_counters[hero1, hero2]
                counter += 1
                
    return [input]

bom1 = [106, 77, 96, 7, 101]
bom2 = [93, 27, 14, 49, 99]


print([idToHero[id] for id in bom1])
print([idToHero[id] for id in bom2])

clf.predict_proba(gen_input(bom1, bom2))

#0.74432872
#0.79667151

['ember_spirit', 'lycan', 'centaur', 'earthshaker', 'skywrath_mage']
['slark', 'shadow_shaman', 'pudge', 'dragon_knight', 'bristleback']


array([[ 0.4253781,  0.5746219]])

In [162]:
bom1 = []
bom2_correct = [93, 27, 14, 49, 99]
bom2 = []

def add_new_hero(bom1, bom2):
    i = len(bom1)
    bom2.append(bom2_correct[i])
    
    
    bom1.append(-1)
    
    best_hero = -1
    best_chance = 0
    
    for hero in range(121):
        hero_name = idToHero.get(hero, "undefined")
        if hero in bom1 or hero in bom2 or hero_name == "undefined": continue
            
        bom1[i] = hero
        input = gen_input(bom1, bom2)
        win_chance = clf.predict_proba(input)[0][1]
            
        if win_chance > best_chance:
            best_hero = hero
            best_chance = win_chance
    
    bom1[i] = best_hero
    
add_new_hero(bom1, bom2)
add_new_hero(bom1, bom2)
add_new_hero(bom1, bom2)
add_new_hero(bom1, bom2)
add_new_hero(bom1, bom2)
print([idToHero.get(hero, "undefined") for hero in bom1])
            

# input = gen_input(bom1, bom2)
# win_chance = clf.predict_proba(input)[0][1]
# print('CHANCE: ', win_chance)

# # best = []


# for hero in range(121):
#     if hero in bom1 or hero in bom2: continue
#     bom1[3] = hero
#     input = gen_input(bom1, bom2)
#     win_chance = clf.predict_proba(input)[0][1]
#     hero_name = idToHero.get(hero, "undefined")
#     if hero_name != "undefined":
#         best.append((win_chance, hero_name))
    
# best.sort()
# print(best[-10:])
# print([idToHero[id] for id in bom1])
# print([idToHero[id] for id in bom2])

['tiny', 'puck', 'lone_druid', 'oracle', 'centaur']


In [ ]:
def print_heroes(match):
    rad = [id for (id, team) in enumerate(match) if team == 1]
    dire = [id for (id, team) in enumerate(match) if team == -1]
    print('Rad:', [idToHero[id] for id in rad])
    print('Dire:', [idToHero[id] for id in dire])

In [83]:
m = np.loadtxt('dota_recent_new.txt', dtype=np.int64)
# m = m[ : 50000]
# print(m)

In [84]:
def compute_synnergy(lines):
    matches_pairs = np.zeros((121, 121))
    wins_pairs = np.zeros((121, 121)) 
    wr_pairs = np.zeros((121, 121))

    for line in lines:
        rad_wins = line[1] == 1
        dire_wins = not rad_wins
        heroes = line[3:]
        rad = heroes[:5]
        dire = heroes[5:]

        for team in [rad, dire]:
            for hero1 in team:
                for hero2 in team:
                    matches_pairs[hero1, hero2] += 1

                    if rad_wins and hero1 in rad:
                        wins_pairs[hero1, hero2] += 1 
                    elif dire_wins and hero2 in dire:
                        wins_pairs[hero1, hero2] += 1

    matches_pairs[matches_pairs == 0] = 1
    wr_pairs = wins_pairs / matches_pairs
    return wr_pairs

In [ ]:
def syn():
    syn_table = np.zeros((121, 121))
    
    for hero1 in range(121):
        for hero2 in range(121):
            syn_table[hero1, hero2] = wr_pairs[hero1, hero2] - (wr_pairs[hero1, hero1] + wr_pairs[hero2, hero2]) / 2
            
    return syn_table

syn_table = syn()
print(syn_table)

In [ ]:
heroes = [(syn_table[hero1, hero2], (idToHero.get(hero1, "undefined"), idToHero.get(hero2, "undefined"))) for hero1 in range(121) for hero2 in range(121) if hero1 < hero2]
heroes.sort()
print(heroes[-10:])


# def choose_best(table):
#     best_heroes = [(-1,-1)
#     val = 0
    
#     for hero1 in range(121):
#         for hero2 in range(121):
#             if table[hero1, hero2] > val:
#                 val = table[hero1, hero2]
#                 best_heroes = (hero1, hero2)
            
#     return best_heroes

# hero1, hero2 = choose_best(syn_table)
# print(idToHero[hero1],idToHero[hero2])

In [85]:
def compute_counters(lines):
    matches_counter = np.zeros((121, 121))
    wins_counter = np.zeros((121, 121)) 
    wr_counter = np.zeros((121, 121))

    for line in lines:
        rad_wins = line[1] == 1
        dire_wins = not rad_wins
        heroes = line[3:]
        rad = heroes[:5]
        dire = heroes[5:]

        for hero1 in rad:
            for hero2 in dire:
                matches_counter[hero1, hero2] += 1
                matches_counter[hero2, hero1] += 1
                if rad_wins: wins_counter[hero1, hero2] += 1
                else: wins_counter[hero2, hero1] += 1

    wins_counter[matches_counter == 0] = 1
    matches_counter[matches_counter == 0] = 1
    wr_counter = wins_counter / matches_counter
    return wr_counter

In [86]:
#np.random.shuffle(m)
factor = 1
train_size = int(len(m) * factor)
wr_pairs = compute_synnergy(m[: train_size])
wr_counters = compute_counters(m[ : train_size])

In [ ]:
print(wr_counters[50, 63])

In [150]:
def format_data(lines):
#     inputs = np.zeros((len(lines), 121))
    inputs = np.zeros((len(lines), 121+75))
    outputs = np.zeros((len(lines)))

    for i in range(len(lines)):
        counter = 0
        
        line = lines[i]
        input = inputs[i]
        outputs[i] = line[1]
        game_type = line[2]
        
        heroes = line[3:]
        rad = heroes[:5]
        dire = heroes[5:]
        
        for hero in rad:
            input[hero] = 1

 
        for hero in dire:
            input[hero] = -1
            
        counter = 121    
        
        for hero1 in rad:
            for hero2 in rad:
                input[counter] = wr_pairs[hero1, hero2]
                counter += 1
              
  
        for hero1 in dire:
                for hero2 in dire:
                    input[counter] = -wr_pairs[hero1, hero2]
                    counter += 1

        for hero1 in rad:
                for hero2 in dire:
                    input[counter] = wr_counters[hero1, hero2]
                    counter += 1


    return (inputs, outputs)

In [151]:

inputs, outputs = format_data(m)
#print(len(inputs))
print(inputs[0])


[ 0.         -1.          0.          0.         -1.          1.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          1.
  1.         -1.          0.          0.          0.         -1.          0.
  0.          0.          0.          0.         -1.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.

In [ ]:
match = inputs[10]
#match[35] = 0
print(len([v for v in match if v <= -1]))

In [ ]:
print(inputs.shape)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adagrad, Adam
from keras import regularizers

model = Sequential()

# limit = 20000
limit = len(inputs)
train_size = int(limit * 0.9)
test_size = limit - train_size
train_in = inputs[ : train_size]
train_out = outputs[: train_size]
test_in = inputs[train_size : limit]
test_out = outputs[train_size : limit]

model.add(Dense(10000, activation='relu', input_dim=len(inputs[0])))
model.add(Dropout(0.4))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

sgd = SGD(lr=0.05)
model.compile(loss='binary_crossentropy',
              #optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
#               optimizer='adadelta',
              optimizer=sgd,
              metrics=['binary_accuracy'])


model.fit(train_in, train_out,
          epochs=20,
          validation_data=(test_in, test_out))


result = model.evaluate(test_in, test_out)
print(result[1])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200, max_features=11, min_samples_split=2, max_depth=None, n_jobs=8)

In [152]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()

In [ ]:
from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(5, weights='distance', n_jobs=8)

In [ ]:
print()
# from sklearn import svm
# clf = svm.SVC()
# clf.fit(data['train']['in'], data['train']['out'])  

In [41]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=500)

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=2000, learning_rate=1.0, max_depth=1, random_state=0)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

clf = BaggingClassifier(LogisticRegression(),
                             max_samples=0.5, max_features=0.5)

In [ ]:
from sklearn.ensemble import VotingClassifier

clf = VotingClassifier(estimators=[('for', clf_forest), ('naive', clf_naive), ('log', clf_log)], voting='soft', weights=[1,2,2])

In [ ]:
from xgboost import XGBClassifier

clf = XGBClassifier()

In [ ]:
class Baseline:
    def fit(inputs, outputs):
        pass
    
    def _pred(self, input):
        rad = []
        dire = []
        
        for hero_id, val in enumerate(input):
            if val == 1: rad.append(hero_id)
            elif val == -1: dire.append(hero_id)
          
#         sinergy_rad = np.average([wr_pairs[hero, hero] for hero in rad])
#         sinergy_dire = np.average([wr_pairs[hero, hero] for hero in dire])
        
        sinergy_rad = np.average([wr_pairs[hero1, hero2] for hero2 in rad for hero1 in rad])
        sinergy_dire = np.average([wr_pairs[hero1, hero2] for hero2 in dire for hero1 in dire])
        
        #print(sinergy_dire)
        
        return 1 if sinergy_rad > sinergy_dire else 0
    
    def predict(self, inputs):
        return [self._pred(input) for input in inputs]

In [ ]:
x = Baseline()

pred = x.predict(inputs[train_size:])
correct = outputs[train_size:]

right = 0

for p,c in zip(pred,correct):
    if p == c:
        right += 1

print(right / len(pred))

In [ ]:
vals = [74.4, 69.5, 63.5, 69.0, 74.5, 52.3, 71.7, 68.6, 59.2]
print([0.93 * val for val in vals])

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(cv=5, n_jobs=8)

In [153]:
from sklearn.metrics import accuracy_score

train_size = int(len(inputs) * 0.9)
test_size = len(inputs) - train_size

clf = clf.fit(inputs[:train_size], outputs[:train_size])
preds = clf.predict(inputs[train_size:])
correct = outputs[train_size:]


print(accuracy_score(correct, preds))

0.6607


In [ ]:
import pandas as pd
import numpy as np

heroes = pd.read_csv('dota-2-match-prediction-master/data/MatchOverviewTraining.csv').values
df_extra = pd.read_csv('dota-2-match-prediction-master/data/MatchDetail.csv')

In [ ]:
print(heroes)

In [ ]:
def gen_data(heroes, df_extra):
    matches = np.zeros((10000, 121), dtype=int)
    out = np.zeros((10000), dtype=int)
    
    for (i, line) in enumerate(heroes):
        out[i] = 1 if line[-1] else 0
        match_id = line[0]
        heroes = line[1:]
        rad = heroes[:5]
        dire = heroes[5:-1]
        
        for hero in rad:
            matches[i][hero] = 1
            
        for hero in dire:
            matches[i][hero] = -1
            
        
#         counter = 121
        
#         exp_gold = df_extra.loc[df_extra['match_id'] == match_id].values
#         for (j, data) in enumerate(exp_gold):
#             matches[i][counter + 2*j] = data[4]
#             matches[i][counter + 2*j + 1] = data[14]
            
    return matches, out
        
        
        

In [ ]:
input, output = gen_data(heroes, df_extra)

In [ ]:
print(input[0])

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adagrad, Adam
from keras import regularizers

model = Sequential()

train_size = int(len(input) * 0.9)
test_size = len(input) - train_size
train_in = input[ : train_size]
train_out = output[: train_size]
test_in = input[train_size : ]
test_out = output[train_size : ]

model.add(Dense(100, activation='relu', input_dim=len(input[0])))
#model.add(Dropout(0.1))
# model.add(Dense(100, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
#model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

#sgd = SGD(lr=0.05)
model.compile(loss='binary_crossentropy',
              #optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              optimizer='adadelta',
              #optimizer=sgd,
              metrics=['binary_accuracy'])

model.fit(train_in, train_out,
          epochs=200,
          validation_data=(test_in, test_out))

result = model.evaluate(test_in, test_out)

print(result[1])

In [ ]:
from xgboost import XGBClassifier

clf = XGBClassifier()

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()

In [ ]:
#print(input[0])
clf.predict_proba([input[1]])

In [ ]:
from sklearn.metrics import accuracy_score


factor = 0.9
train_size = int(len(inputs) * factor)
test_size = len(inputs) - train_size
train_in, train_out, test_in, test_out = inputs[:train_size], outputs[:train_size], inputs[train_size:], outputs[train_size:]


clf = clf.fit(train_in, train_out)
end = time.time()


preds = clf.predict(test_in)
correct = test_out

print(accuracy_score(correct, preds))

In [ ]:
match = inputs[0]
print_heroes(match)
print(match)

In [ ]:
match[114] = 0

In [ ]:
match[22] = 0
print_heroes(match)
print(match)

In [ ]:
best = 0
best_chance = 0

for i in range(120):
    if match[i] != 0: continue
    match[i] = 1
    chance_win = clf.predict_proba([match])[0][1]
    
    if chance_win > best_chance:
        best_chance = chance_win
        best = i
        
    print('chance: %f hero: %s' % (chance_win, idToHero.get(i, "N/A")))
    match[i] = 0

print('BEST:', idToHero[best])
match[0] = 0
print_heroes(match)

# print(len([v for v in match if v != 0]))


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the parameters by cross-validation
# tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                      'C': [1, 10, 100, 1000]},
#                     {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

tuned_parameters = [{'n_estimators': [10, 50, 100], 'max_features': [11, 40, 121]}]

scores = ['precision']

X_train, X_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.5, random_state=0)


clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=5,
                       scoring='%s_macro' % 'precision')
clf.fit(X_train, y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()
